In [1]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import torch

In [2]:
class CreateDataset:
    def __init__(self, device, use_stopword=True):
        
        ### cpu or gpu setting ###
        self.device = device
        
        ### stopword 사용 여부 변수 ###
        self.use_stopword = use_stopword
        
        ### spacy library를 활용하여 tokenizer에 활용 ###
        self.spacy_de = spacy.load('de_core_news_sm')
        self.spacy_en = spacy.load('en_core_web_sm')
        
        ### spacy에서 제공하는 stop word list ###
        self.de_stopwords = self.spacy_de.Defaults.stop_words
        self.en_stopwords = self.spacy_en.Defaults.stop_words
        
        ### 데이터 셋의 전처리에 관한 부분을 담고 있음 ###
        ### tokenize는 아래의 함수를 활용하며, 시작 토큰에 <sos>를 추가, 끝 토큰엔 <eos>를 추가하고
        ### 소문자로 정규화
        self.SRC = Field(tokenize=self.tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True)
        self.TRG = Field(tokenize=self.tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True)
        
        ### torchtext.data로 부터 de (독일어), en(영어) 데이터 셋을 가져와서
        ### 앞전에 선언해 둔 전처리 함수를 넣어줘서 전처리된 값을 return 받는다
        self.train_data, self.valid_data, self.test_data = Multi30k.splits(exts=('.de', '.en'), fields=(self.SRC, self.TRG))
        
        ### vocabulary를 만드는 과정으로 train_data에 있는 단어들을 가지고
        ### 2번 이상 등장한 단어들로 vocab을 만든다.
        self.SRC.build_vocab(self.train_data, min_freq = 2)
        self.TRG.build_vocab(self.train_data, min_freq = 2)
    
    ### 전처리에 들어갈 tokenize 함수들
    def tokenize_de(self, text)
        if self.use_stopword:
            return [tok.text for tok in self.spacy_de.tokenizer(text)]
        return [tok.text for tok in self.spacy_de.tokenizer(text) if tok not in self.de_stopwords]
    def tokenize_en(self, text):
        if self.use_stopword:
            return [tok.text for tok in self.spacy_en.tokenizer(text)]
        return [tok.text for tok in self.spacy_de.tokenizer(text) if tok not in self.en_stopwords]

    ### 최종적으로 iterator에 데이터 셋을 담아서 iterator를 반환
    ### 함수에선 해당 iterator로 부터 데이터를 뽑아내어 사용할 것
    def get_iterator(self, batch_size):
        train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
            (self.train_data, self.valid_data, self.test_data), 
            batch_size = batch_size, 
            device = self.device)
        return train_iterator, valid_iterator, test_iterator

In [3]:
# if __name__ == '__main__':
#     device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     dataset = CreateDataset(device, use_stopword=False)
#     train, valid, test = dataset.get_iterator(2)
#     for x in train:
#         print(x.src)
#         print(x.trg)
#         break
#     for x in valid:
#         print(x.src)
#         print(x.trg)
#         break
#     for x in test:
#         print(x.src)
#         print(x.trg)
#         break